In [1]:
import torch
from torch.utils.cpp_extension import load

if torch.cuda.is_available():
    device = torch.device("cuda")

In [2]:
ultmul = load(name='ultmul', sources=['ultMul_cuda.cpp', 'ultMul_cuda_kernel.cu'])

import ultmul as um
um.ultmul

<function ultmul.PyCapsule.ultmul>

In [6]:
import sklearn.metrics.pairwise as sk_dist
torch.manual_seed(11000)
test = torch.randn(20,10).float() # number of samples have to smaller than sqrt(512) need to fix
euc_dist = torch.from_numpy(sk_dist.euclidean_distances(test.numpy())).to(device).float()
print(euc_dist.size())
euc_dist

torch.Size([20, 20])


tensor([[0.0000, 3.8651, 3.7960, 3.9651, 3.3758, 4.6612, 3.3628, 3.8138, 6.1340,
         3.3289, 4.5001, 2.6686, 4.0458, 4.9201, 5.2727, 4.6603, 3.6126, 4.8607,
         4.7274, 4.1477],
        [3.8651, 0.0000, 3.6884, 4.8299, 3.9495, 4.0814, 3.3933, 4.2135, 5.6796,
         4.5118, 5.0394, 2.9462, 3.5917, 3.9574, 4.3291, 3.9077, 4.0505, 3.1202,
         5.4969, 3.6318],
        [3.7960, 3.6884, 0.0000, 5.5954, 4.9571, 4.7856, 3.5409, 4.1538, 5.7041,
         4.4075, 4.7869, 3.0711, 2.8700, 4.7967, 5.0666, 4.4211, 3.9446, 3.4845,
         4.7059, 4.2531],
        [3.9651, 4.8299, 5.5954, 0.0000, 3.5860, 5.3707, 4.1983, 5.4678, 5.2637,
         3.2713, 3.8631, 3.8031, 4.2567, 4.4062, 5.4859, 4.7221, 4.1189, 5.2226,
         5.7114, 5.3516],
        [3.3758, 3.9495, 4.9571, 3.5860, 0.0000, 3.2115, 2.4182, 3.7872, 5.2665,
         3.6395, 3.9530, 3.2514, 4.0487, 4.9870, 4.3799, 4.6588, 3.2940, 3.9427,
         5.0325, 4.7593],
        [4.6612, 4.0814, 4.7856, 5.3707, 3.2115, 0.0000, 3.3

In [4]:
#euc_dist
import time
torch.cuda.synchronize()   
start = time.clock()
u_test = um.ultmul(euc_dist, euc_dist)
torch.cuda.synchronize() 
end = time.clock()
tu = end-start
print(tu)

0.0002949999999994901


In [5]:
D_sub = um.single_hclust(euc_dist, 3)
print(D_sub)

[tensor([4]), tensor([1., 1., 1., 1., 1., 1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        3., 1.], device='cuda:0')]


In [5]:
start = time.clock()
uclb = um.clusterability(euc_dist)
end = time.clock()
tc = end-start
print(tc)
print(uclb)

0.0020690000000005426
[tensor([6]), tensor([[0.0000, 2.9462, 2.8700, 3.2713, 2.7640, 3.2115, 2.7640, 3.7872, 2.9481,
         2.6686, 2.9481, 2.6686, 2.6686, 3.1963, 2.6686, 2.6686, 2.6686, 3.1202,
         3.9626, 3.0916],
        [2.9462, 0.0000, 2.9462, 3.2713, 2.9462, 3.2115, 2.9462, 3.7872, 2.9481,
         2.9462, 2.9481, 2.9462, 2.9462, 3.1963, 2.9462, 2.9462, 2.9462, 3.1202,
         3.9626, 3.0916],
        [2.8700, 2.9462, 0.0000, 3.2713, 2.8700, 3.2115, 2.8700, 3.7872, 2.9481,
         2.8700, 2.9481, 2.8700, 2.8700, 3.1963, 2.8700, 2.8700, 2.8700, 3.1202,
         3.9626, 3.0916],
        [3.2713, 3.2713, 3.2713, 0.0000, 3.2713, 3.2713, 3.2713, 3.7872, 3.2713,
         3.2713, 3.2713, 3.2713, 3.2713, 3.2713, 3.2713, 3.2713, 3.2713, 3.2713,
         3.9626, 3.2713],
        [2.7640, 2.9462, 2.8700, 3.2713, 0.0000, 3.2115, 2.4182, 3.7872, 2.9481,
         2.7640, 2.9481, 2.7640, 2.7640, 3.1963, 2.7640, 2.7640, 2.7640, 3.1202,
         3.9626, 3.0916],
        [3.2115, 3.2115,

In [6]:
torch.cuda.synchronize()  
start = time.clock()
m_test = torch.matmul(euc_dist, euc_dist)
torch.cuda.synchronize()
end = time.clock()
tm = end-start
print(tm)

0.30459700000000023


In [17]:
print(torch.max(euc_dist))

tensor(6.6564, device='cuda:0')


torch.Size([20, 20])

In [11]:
from sklearn.cluster import AgglomerativeClustering
import numpy as np
X = euc_dist.to('cpu').numpy()
clustering = AgglomerativeClustering(n_clusters = 3, affinity = 'precomputed', linkage='single').fit(X)
print(clustering.labels_)

[0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 1 0]


In [12]:
from sklearn.metrics.cluster import normalized_mutual_info_score
normalized_mutual_info_score(clustering.labels_, D_sub[1].to('cpu').numpy())

1.0